<a href="https://colab.research.google.com/github/HeshanDissanayake/Convolutional_Auto_encoder/blob/main/Convolutional_Auto_encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

lr = 0.001
dashes = '---------------------------'
batchSize = 50
def autoEncoder(input_dim, kernal_dims, n_kernals):

    """ :param input_dim: dimensions of the input
        :param kernal_dim: list of filter dimensoins
        :param n_kernal:   list of the number of filters in each layer
        :return: output tensor
    """

    X = tf.placeholder(tf.float32, input_dim, name= 'inputs')
    input = X

    layer_dims = [] # a list to store the dimensions of layers to use in decoder construction
    print('Constructing the auto encoder')
    for layer_idx, kernal in enumerate(kernal_dims):

        layer_dims.append(input.shape)
        in_chanels = input.get_shape().as_list()[3]
        out_chanels = n_kernals[layer_idx]

        print(dashes,'\n','convolution == %dx%dx%d'%(input.shape[1],input.shape[2],input.shape[3]))

        w = tf.Variable(tf.random_uniform([kernal, kernal, in_chanels, out_chanels], -1/in_chanels, 1/in_chanels))
        b = tf.Variable(tf.zeros([out_chanels]))

        input = tf.add(tf.nn.conv2d(input, w, strides=[1, 2, 2, 1], padding='SAME'),b)
        input = tf.nn.relu(input)
    latent = tf.identity(input, name = 'latent')
    # temp = input.get_shape().as_list()
    # latent = tf.reshape(input,[-1,1,temp[1]*temp[2]*temp[3]], name='latent')

    # print(dashes, '\n', 'convolution == %dx%dx%d' % (input.shape[1], input.shape[2], input.shape[3]))
    # print(dashes, '\n', 'latent space == %dx%d'%(latent.shape[1],latent.shape[2]))


    # input = tf.reshape(latent,[tf.shape(X)[0],temp[1],temp[2],temp[3]])

    layer_dims.reverse()
    kernal_dims.reverse()
    n_kernals.reverse()

    for layer_idx, kernal in enumerate(kernal_dims):

        in_chanels = input.get_shape().as_list()[3]
        out_chanels = layer_dims[layer_idx].as_list()[3]
        print(dashes,'\n','convolution == %dx%dx%d'%(input.shape[1],input.shape[2],input.shape[3]))


        w = tf.Variable(tf.random_uniform([kernal, kernal, out_chanels, in_chanels],
                                          -1/in_chanels, 1/in_chanels, tf.float32))

        b = tf.Variable(tf.zeros([out_chanels]))
        shape = layer_dims[layer_idx]
        input = tf.nn.conv2d_transpose(input, w, [tf.shape(X)[0], shape[1], shape[2], shape[3]], strides=[1, 2, 2, 1], padding='SAME')
        input = tf.add(input,b)
        input = tf.nn.relu(input)

    input = tf.identity(input, name = 'output')

    cost =tf.reduce_sum(tf.square(X - input))
    return {"input": X, "output":input, "cost":cost}




data = np.load('data.npy')/255
im = (data[30,:,:,:]*255).astype(np.int)

n = np.zeros((1,im.shape[0],im.shape[1], 3))
n[0,:,:,:] = data[30,:,:,:]


plt.imshow(im)
plt.pause(0.001)

Aencoder = autoEncoder([None,n.shape[1],n.shape[2],3],list([10, 5, 5, 5]),list([10, 20, 30, 1]))

optimizer = tf.train.AdamOptimizer(learning_rate = 0.0001).minimize(Aencoder["cost"])
ses= tf.Session()
ses.run(tf.global_variables_initializer())

saver = tf.train.Saver()
for i in range(1000000):
    for j in range(0,9000, 50):
        miniBatch = data[j:j+50,:,:,:]
        ses.run(optimizer, feed_dict={Aencoder['input']: miniBatch})
        
    print('ecpoc',i,'cost is:',ses.run(Aencoder['cost'],feed_dict={Aencoder['input']:data[9000:9200,:,:,:]}))
    if i%100 == 0:
        xx = ses.run(Aencoder["output"], feed_dict={Aencoder["input"]: n})
        print('Saving the model')
        saver.save(ses, './AE')
        plt.imshow((xx[0, :, :, :]*255).astype(np.int))
        print((xx[0, :, :, :]*255).astype(np.int))
        plt.pause(0.001)




In [ ]:
im = cv2.imread('/content/me.jpg')
print(im)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!cp -R checkpoint /content/gdrive/My\ Drive/checkpoint
!cp -R AE.data-00000-of-00001	 /content/gdrive/My\ Drive/AE.data-00000-of-00001	
!cp -R AE.meta  /content/gdrive/My\ Drive/AE.meta 
!cp -R AE.index  /content/gdrive/My\ Drive/AE.index 


In [ ]:
!cp -R  /content/gdrive/My\ Drive/data.npy data.npy

In [ ]:
!rm AE.index AE.meta AE.data-00000-of-00001 checkpoint

In [ ]:
!ls

sample_data


In [ ]:
n[0,:,:,:] = data[30,:,:,:]/255
n = n.astype(np.float32)
print(n.dtype)
sess = tf.Session()

meta = tf.train.import_meta_graph('./AE.meta')
meta.restore(sess, ('./AE'))
print("buf1")

graph = tf.get_default_graph()
input = graph.get_tensor_by_name("inputs:0")
output = graph.get_tensor_by_name("output:0")
print(input)

x = sess.run(output, feed_dict={input : n})
x = (x*1000).astype(np.int)
print(x[0, :, :, :])
plt.imshow(x[0, :, :, :])
plt.pause(10)
